In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import sklearn

## 데이터 로드 및 train test로 분할

In [8]:
# df_kookmin = pd.read_csv('/Users/lhs/Desktop/GitHub/MLT/EDA/non_financial/crawling/news/oasis_kookmin.csv') #.drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link'])
# df_kyunghyan = pd.read_csv('/Users/lhs/Desktop/GitHub/MLT/EDA/non_financial/crawling/news/oasis_kyunghyan.csv') #.drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link'])
df_naver = pd.read_csv('/Users/lhs/Desktop/GitHub/MLT/EDA/non_financial/crawling/news/oasis_navernews.csv')


# df_kookmin = pd.read_csv('D:/Documents/GitHub/MLT/EDA/non_financial/crawling/news/oasis_kookmin.csv') #.drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link'])
# df_kyunghyan = pd.read_csv('D:/Documents/GitHub/MLT/EDA/non_financial/crawling/news/oasis_kyunghyan.csv') #.drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link'])


In [9]:
# df = pd.concat([df_kookmin, df_kyunghyan]).drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','links'])
# df = df.set_index(['dates'])


In [10]:
df_naver = df_naver.drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','links'])

In [11]:
df_naver

,titles,dates,contents
0,"[오아시스마켓, 맞춤형 영상 콘텐츠 지원에 소상공인 매출 ‘쑥쑥’]",2022.09.30. 오전 10:22,"[\n""V 커머스 지원 사업으로 2~400배 매출 증가""오아시스마켓이 중소기업유통센..."
1,"[기업가치 1兆 ‘오아시스마켓’, 상장 예심 청구 “올해 IPO”]",2022.09.08. 오후 2:09,[\n마켓컬리 이어 두번째 식품 이커머스 상장생협 출신이 2011년 설립...유일한...
2,"[오아시스마켓, ‘돌나라 오아시스’ 연관 의혹에 ""전혀 관련 없어""]",2022.09.13. 오후 5:11,"[\n""명예훼손·허위사실 유포에 강력 대응 방침""새벽배송 전문 업체 오아시스마켓이 ..."
3,"[[고고IPO] 오아시스마켓, 경기불황에도 IPO 속도 ‘자신감’]",2022.09.09. 오전 10:58,[\n기업들이 뉴노멀 시대에 대응하며 성장을 지속하고 있다. 신사업 투자를 위한 자...
4,[급성장한 '컬리' 고수익의 '오아시스'…동시 상장하면 누가 흥행할까],2022.09.13. 오전 5:30,[\n\n\n\n\n컬리에 이어 오아시스마켓까지 '연내 상장'을 목표로 상장 준비에...
...,...,...,...
1228,[위드 코로나…오프라인 유통株 주목],2021.10.27. 오후 3:57,[\n신세계·현대百·호텔신라 '톱픽'내년은 유통주의 격변기가 될 전망이다. ‘위드 ...
1229,[대전 6개 기업 IT 기술력으로 해외시장 공략 나선다],2021.10.27. 오후 12:53,[\n\n\n\n\n2019 MWC(Mobile World Congress) LA ...
1230,[[ET] 일주일 앞으로 다가온 ‘위드 코로나’…유통주가 대세?],2021.10.25. 오후 6:10,[\n■ 프로그램명 : 통합뉴스룸ET■ 코너명 : ET WHY?■ 방송시간 : 10...
1231,"[딜리버리히어로, 10분 배송 '고릴라스'에 2770억원 투자]",2021.10.22. 오후 3:35,[\n\n\n\n\n독일 딜리버리히어로가 퀵커머스 스타트업 고릴라스에 2억3500만...


In [12]:
for i in range(1233):
    df_naver.dates[i] =df_naver.dates[i][:10]

In [13]:
df_naver

,titles,dates,contents
0,"[오아시스마켓, 맞춤형 영상 콘텐츠 지원에 소상공인 매출 ‘쑥쑥’]",2022.09.30,"[\n""V 커머스 지원 사업으로 2~400배 매출 증가""오아시스마켓이 중소기업유통센..."
1,"[기업가치 1兆 ‘오아시스마켓’, 상장 예심 청구 “올해 IPO”]",2022.09.08,[\n마켓컬리 이어 두번째 식품 이커머스 상장생협 출신이 2011년 설립...유일한...
2,"[오아시스마켓, ‘돌나라 오아시스’ 연관 의혹에 ""전혀 관련 없어""]",2022.09.13,"[\n""명예훼손·허위사실 유포에 강력 대응 방침""새벽배송 전문 업체 오아시스마켓이 ..."
3,"[[고고IPO] 오아시스마켓, 경기불황에도 IPO 속도 ‘자신감’]",2022.09.09,[\n기업들이 뉴노멀 시대에 대응하며 성장을 지속하고 있다. 신사업 투자를 위한 자...
4,[급성장한 '컬리' 고수익의 '오아시스'…동시 상장하면 누가 흥행할까],2022.09.13,[\n\n\n\n\n컬리에 이어 오아시스마켓까지 '연내 상장'을 목표로 상장 준비에...
...,...,...,...
1228,[위드 코로나…오프라인 유통株 주목],2021.10.27,[\n신세계·현대百·호텔신라 '톱픽'내년은 유통주의 격변기가 될 전망이다. ‘위드 ...
1229,[대전 6개 기업 IT 기술력으로 해외시장 공략 나선다],2021.10.27,[\n\n\n\n\n2019 MWC(Mobile World Congress) LA ...
1230,[[ET] 일주일 앞으로 다가온 ‘위드 코로나’…유통주가 대세?],2021.10.25,[\n■ 프로그램명 : 통합뉴스룸ET■ 코너명 : ET WHY?■ 방송시간 : 10...
1231,"[딜리버리히어로, 10분 배송 '고릴라스'에 2770억원 투자]",2021.10.22,[\n\n\n\n\n독일 딜리버리히어로가 퀵커머스 스타트업 고릴라스에 2억3500만...


In [14]:
# df['dates'] = df['dates'].str.replace('.','-')
df_naver['dates'] = df_naver['dates'].str.replace(' ', '')

In [15]:
for i in range(44):
    print(df_naver.dates.value_counts()[(i-1)*30:i*30])

Series([], Name: dates, dtype: int64)
2022.08.11    32
2022.08.31    31
2022.08.30    26
2022.07.10    22
None          20
2022.03.31    17
2022.07.06    17
2022.04.15    16
2022.07.27    16
2022.03.28    15
2022.07.26    15
2022.06.02    15
2022.06.07    15
2022.05.09    14
2021.10.29    12
2022.09.01    12
2022.07.21    12
2022.08.25    12
2021.10.28    11
2022.01.04    11
2022.03.29    11
2022.08.09    11
2022.08.22    11
2022.08.29    11
2022.05.13    10
2022.05.31    10
2022.08.17    10
2022.09.21    10
2022.08.28    10
2021.10.27     9
Name: dates, dtype: int64
2022.01.24    9
2022.04.29    9
2021.10.26    9
2021.12.29    9
2022.05.27    9
2022.04.28    9
2022.03.30    8
2022.04.01    8
2022.02.07    8
2022.07.11    8
2021.11.08    7
2022.03.18    7
2021.11.23    7
2022.04.16    7
2021.11.02    7
2022.09.02    7
2022.04.12    7
2022.01.02    7
2022.03.21    7
2022.07.18    7
2022.07.22    7
2022.06.19    6
2021.10.07    6
2022.09.05    6
2021.10.31    6
2022.04.21    6
2022.08.15

In [16]:
df_naver = df_naver.sort_values(by=['dates'], axis=0).reset_index().drop(columns=['index'])#.drop(columns=['level_0'])
df_naver

,titles,dates,contents
0,[[톡톡 이상품] 한빛코리아 / 다지워 네이처 클렌징 솝],2021.10.05,[\n마스카라까지 지우는 세정비누\n\n\n\n 피부에 자극을 주지 않고 각종 화장...
1,[[톡톡 이상품] 한빛코리아 / 다지워 네이처 클렌징 솝],2021.10.05,[\n마스카라까지 지우는 세정비누\n\n\n\n 피부에 자극을 주지 않고 각종 화장...
2,[[격동의 유통가④]편의점 성장 정체…미래는 근거리 쇼핑],2021.10.07,[\n기사내용 요약매장은 늘고 점포당 매출은 매월 줄고 있어코로나19로 마트 대신 ...
3,"[""파괴적 커머스 시대, 데이터 경쟁으로 전환"" [제14회 유통혁신포럼]]",2021.10.07,"[\n정연승 유통학회장 기조 강연""유통산업 카테고리 경계 사라져"" \n\n\n\n ..."
4,"[""파괴적 커머스 시대, 데이터 경쟁으로 전환"" [제14회 유통혁신포럼]]",2021.10.07,"[\n정연승 유통학회장 기조 강연""유통산업 카테고리 경계 사라져"" \n\n\n\n ..."
...,...,...,...
1228,[],None,[]
1229,[],None,[]
1230,[],None,[]
1231,[],None,[]


In [17]:
ss=df_naver[df_naver['dates'].str.contains('None')].index
ss

Int64Index([1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223,
            1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232],
           dtype='int64')

In [18]:
df_naver = df_naver.drop(ss)

In [19]:
df_naver

,titles,dates,contents
0,[[톡톡 이상품] 한빛코리아 / 다지워 네이처 클렌징 솝],2021.10.05,[\n마스카라까지 지우는 세정비누\n\n\n\n 피부에 자극을 주지 않고 각종 화장...
1,[[톡톡 이상품] 한빛코리아 / 다지워 네이처 클렌징 솝],2021.10.05,[\n마스카라까지 지우는 세정비누\n\n\n\n 피부에 자극을 주지 않고 각종 화장...
2,[[격동의 유통가④]편의점 성장 정체…미래는 근거리 쇼핑],2021.10.07,[\n기사내용 요약매장은 늘고 점포당 매출은 매월 줄고 있어코로나19로 마트 대신 ...
3,"[""파괴적 커머스 시대, 데이터 경쟁으로 전환"" [제14회 유통혁신포럼]]",2021.10.07,"[\n정연승 유통학회장 기조 강연""유통산업 카테고리 경계 사라져"" \n\n\n\n ..."
4,"[""파괴적 커머스 시대, 데이터 경쟁으로 전환"" [제14회 유통혁신포럼]]",2021.10.07,"[\n정연승 유통학회장 기조 강연""유통산업 카테고리 경계 사라져"" \n\n\n\n ..."
...,...,...,...
1208,"[""투자 늘렸는데 이제와서…"" 퀵커머스마저 규제하나 '초긴장']",2022.09.27,[\n■국감 오르는 유통업 새 먹거리e커머스 맞서 경쟁력 강화 위해쿠팡이츠마트 등 ...
1209,"[""투자 늘렸는데 이제와서…"" 퀵커머스마저 규제하나 '초긴장']",2022.09.27,[\n■국감 오르는 유통업 새 먹거리e커머스 맞서 경쟁력 강화 위해쿠팡이츠마트 등 ...
1210,"[""투자 늘렸는데 이제와서…"" 퀵커머스마저 규제하나 '초긴장']",2022.09.27,[\n■국감 오르는 유통업 새 먹거리e커머스 맞서 경쟁력 강화 위해쿠팡이츠마트 등 ...
1211,[국내 이커머스 상장 1호 주인공 누가될까 [격변의 이커머스①]],2022.09.29,"[\n이커머스 IPO 대전, 흑자 기업 오아시스 연내 목표컬리, 증시 부진에 예상 ..."


In [20]:
df_naver = df_naver.drop_duplicates().reset_index().drop(columns=['index'])

In [21]:
df_naver = df_naver.set_index('dates')

In [24]:
df_naver['cc'] = '//'

In [25]:
df_naver

,titles,contents,cc
dates,,,
2021.10.05,[[톡톡 이상품] 한빛코리아 / 다지워 네이처 클렌징 솝],[\n마스카라까지 지우는 세정비누\n\n\n\n 피부에 자극을 주지 않고 각종 화장...,//
2021.10.07,[[격동의 유통가④]편의점 성장 정체…미래는 근거리 쇼핑],[\n기사내용 요약매장은 늘고 점포당 매출은 매월 줄고 있어코로나19로 마트 대신 ...,//
2021.10.07,"[""파괴적 커머스 시대, 데이터 경쟁으로 전환"" [제14회 유통혁신포럼]]","[\n정연승 유통학회장 기조 강연""유통산업 카테고리 경계 사라져"" \n\n\n\n ...",//
2021.10.07,"[""비대면 소비로 새벽배송 급증 물류센터 확대 전국 서비스 추진"" [제14회 유통혁...","[\n강연 K유니콘의 미래전략김수희 오아시스마켓 대외법무이사 \n\n\n\n ""신...",//
2021.10.07,[2021년 9월 프랜차이즈 정보공개서 신규등록 333개 명단 공개],[\n2021년 9월 신규정보공개서 등록브랜드가 333개로 나타났다. 정보공개서 등...,//
...,...,...,...
2022.09.23,"[WCP 흥행 실패에 컬리, 케이뱅크 상장 시기 '안갯속']",[\n심사 승인 났는데 IPO 감감무소식골프존카운티는 내년으로 연기컬리도 오아시스 ...,//
2022.09.26,"[[먹거리+IT] 네이처랩 오경주 대표, “맛있고 건강한 주스 추천 서비스를 개발합...",[\n\t\t\t서울먹거리창업센터는 서울시가 국내 최초로 설립한 농식품 분야 특화 ...,//
2022.09.27,"[""투자 늘렸는데 이제와서…"" 퀵커머스마저 규제하나 '초긴장']",[\n■국감 오르는 유통업 새 먹거리e커머스 맞서 경쟁력 강화 위해쿠팡이츠마트 등 ...,//


In [26]:
df_naver['titles'] = df_naver['titles'] + df_naver['cc'] 
df_naver['contents'] = df_naver['contents'] + df_naver['cc'] 

In [29]:
df_naver = df_naver.drop(columns=['cc'])
df_naver

,titles,contents
dates,,
2021.10.05,[[톡톡 이상품] 한빛코리아 / 다지워 네이처 클렌징 솝]//,[\n마스카라까지 지우는 세정비누\n\n\n\n 피부에 자극을 주지 않고 각종 화장...
2021.10.07,[[격동의 유통가④]편의점 성장 정체…미래는 근거리 쇼핑]//,[\n기사내용 요약매장은 늘고 점포당 매출은 매월 줄고 있어코로나19로 마트 대신 ...
2021.10.07,"[""파괴적 커머스 시대, 데이터 경쟁으로 전환"" [제14회 유통혁신포럼]]//","[\n정연승 유통학회장 기조 강연""유통산업 카테고리 경계 사라져"" \n\n\n\n ..."
2021.10.07,"[""비대면 소비로 새벽배송 급증 물류센터 확대 전국 서비스 추진"" [제14회 유통혁...","[\n강연 K유니콘의 미래전략김수희 오아시스마켓 대외법무이사 \n\n\n\n ""신..."
2021.10.07,[2021년 9월 프랜차이즈 정보공개서 신규등록 333개 명단 공개]//,[\n2021년 9월 신규정보공개서 등록브랜드가 333개로 나타났다. 정보공개서 등...
...,...,...
2022.09.23,"[WCP 흥행 실패에 컬리, 케이뱅크 상장 시기 '안갯속']//",[\n심사 승인 났는데 IPO 감감무소식골프존카운티는 내년으로 연기컬리도 오아시스 ...
2022.09.26,"[[먹거리+IT] 네이처랩 오경주 대표, “맛있고 건강한 주스 추천 서비스를 개발합...",[\n\t\t\t서울먹거리창업센터는 서울시가 국내 최초로 설립한 농식품 분야 특화 ...
2022.09.27,"[""투자 늘렸는데 이제와서…"" 퀵커머스마저 규제하나 '초긴장']//",[\n■국감 오르는 유통업 새 먹거리e커머스 맞서 경쟁력 강화 위해쿠팡이츠마트 등 ...


In [30]:
df_naver = df_naver.groupby('dates').sum()

In [33]:
df_naver['label'] = '-'

In [34]:
df_naver

,titles,contents,label
dates,,,
2021.10.05,[[톡톡 이상품] 한빛코리아 / 다지워 네이처 클렌징 솝]//,[\n마스카라까지 지우는 세정비누\n\n\n\n 피부에 자극을 주지 않고 각종 화장...,-
2021.10.07,"[[격동의 유통가④]편의점 성장 정체…미래는 근거리 쇼핑]//[""파괴적 커머스 시대...",[\n기사내용 요약매장은 늘고 점포당 매출은 매월 줄고 있어코로나19로 마트 대신 ...,-
2021.10.08,[株머니 매니저의 HOT종목]//,[\n\n\n\n\n 씨에스윈드는 풍력발전기를 구성하는 타워 제작 및 설치를 주력 ...,-
2021.10.11,"[김민수 더맘마 대표 ""동네마트와 '상생'이 경쟁력…30분 총알배송 목표""]//",[\n별도 물류센터 없이 동네마트 활용…中 알리바바의 슈퍼마켓 '허만셴셩' 벤치마크...,-
2021.10.13,"[""카카오 보고있나""…진격의 네이버페이, 中 최대 직구몰 손잡았다]//[[주간투자동...",[\n\n\n\n\n/사진=알리익스프레스중국 최대 규모의 해외직구 쇼핑몰에서도 네이...,-
...,...,...,...
2022.09.23,"[WCP 흥행 실패에 컬리, 케이뱅크 상장 시기 '안갯속']//",[\n심사 승인 났는데 IPO 감감무소식골프존카운티는 내년으로 연기컬리도 오아시스 ...,-
2022.09.26,"[[먹거리+IT] 네이처랩 오경주 대표, “맛있고 건강한 주스 추천 서비스를 개발합...",[\n\t\t\t서울먹거리창업센터는 서울시가 국내 최초로 설립한 농식품 분야 특화 ...,-
2022.09.27,"[""투자 늘렸는데 이제와서…"" 퀵커머스마저 규제하나 '초긴장']//",[\n■국감 오르는 유통업 새 먹거리e커머스 맞서 경쟁력 강화 위해쿠팡이츠마트 등 ...,-


In [35]:
df_naver.to_csv('news_label.csv')

# --------------------

## 같은 날짜 한줄에 표시

In [ ]:
df = df.groupby('dates').sum()
df

## null 값 확인

In [ ]:
df.info()

## 특수문자 제거 정규표현식

In [ ]:
df['titles'] = df['titles'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')
df['contents'] = df['contents'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')

In [ ]:
df

## 토큰화

In [ ]:
okt = Okt()


In [ ]:
# 불용어 처리 

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다',
             '\n','\n ','제목','을','로','있다','등']
stopwords

In [ ]:
token = []
for sentence in tqdm(df['contents']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    token.append(stopwords_removed_sentence)
    

In [ ]:
len(token)

In [ ]:
print(token[:1])

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(token)

In [ ]:
print(tokenizer.word_index)

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
from collections import Counter

In [ ]:
# 한 날짜? 한 행당 횟수 추출

for i in range(19):
    vocab = Counter(token[i])
    print(vocab)

In [ ]:
# 전체 중 빈도수 출력

all_word_list = sum(token, [])
vocab = Counter(all_word_list)
print(vocab)

In [ ]:
word_list = list(vocab.keys())
word_list

In [ ]:
word_rank = list(vocab.values())
word_rank

In [ ]:
data = {'word': word_list, 'count' : word_rank}
word_data = pd.DataFrame.from_dict(data)
word_data

In [ ]:
word_data = word_data.sort_values(by=['count'], ascending=False).reset_index().drop(columns=['index'])

In [ ]:
word_data

In [ ]:
for i in range(19):
    print(word_data[ i*10 : (i+1)*10 ])

In [ ]:
word_data.to_csv('sentimental(news)_.csv')

In [ ]:
sentimental = pd.read_csv('sentimental(news).csv').drop(columns=['Unnamed: 0'])
# sentimental = sentimental.fillna('-')
sentimental.insert(2,'label','-')

In [ ]:
# sentimental.to_csv('sentimental(news).csv')

In [ ]:
sentimental

### 추출했던 word_list에서 불용어 추가

In [ ]:
import csv

with open("stopword.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(word_list)

In [ ]:
with open('stopword.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

In [ ]:
stopword_add = sum(data, [])
stopword_add

In [2]:
a = pd.read_csv('38_label_x.csv')

In [3]:
a

,date,title,content,label
0,2020-09-04,[제목 : [오아시스]\n 팬데믹시대에 내마음의 표준을 바꾸는 유통 ],이번 코로나가 유통시장의 판도를 5년이상 앞당겨진 변화를 가져 왔다?매장에서 인테넷...,-
1,2021-04-17,[제목 : [오아시스] 오아시스 매수 했습니다. ][제목 : [오아시스] 오아시스 ...,"오아이스 주식 중 pe가 보유한 물량이 시장에서 움직이는 듯 하네요.어제, 아시는...",-
2,2021-04-19,[제목 : [오아시스] 70만원이면 비싼건가요 ],"30주 70만원에 살려고하는데, 비싼건가요//",-
3,2021-04-22,[제목 : [오아시스] 70만원이면 비싼건가요 ],어디서 70만원에 살수있어요?나도 사고 싶다//,-
4,2021-04-23,[코멘트 : [오아시스] pe가뭐에요? ][제목 : [오아시스] 망설이다가 못 샀네요 ],"pe가뭐에요?//70만원에 살까 말까 고민하고 있는데,팔렸다고 하네요.""빨간색매물검...",-
...,...,...,...,...
81,2022-07-27,[코멘트 : [오아시스] 구라치치말고 아는거 있음 그냥 직접 올려 ],구라치치말고 아는거 있음 그냥 직접 올려//,-
82,2022-08-31,[제목 : [오아시스] 우크라이나와 러시아 ][제목 : [오아시스] 오늘 날짜(20...,덩치 큰 러시아가 누구다 다 이길거라고 했지만 결과는? 길고 짧은건 대봐야 안다고우...,-
83,2022-09-02,[제목 : [오아시스] 이 시기에 오히려 주목해야 할 기업 ],연일 금리가 급등하고 있다. 매출만을 키우기 위해 무한확장을 거듭했던 스타트업과 유...,-
84,2022-09-05,[제목 : [오아시스] 계획된 적자 ],한때 VC와 스타트업들이 힘을 합쳐 매출액을 늘려 시장 주도권을 가져가기위한 전략으...,-
